## Test of the images with Otsu thresholding

In [1]:
import os
from pathlib import Path
from pydantic import BaseSettings
from torch.utils.data import DataLoader

import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import torch
from typing import List, Tuple

In [2]:
cwd = Path(os.getcwd())
os.chdir(str(cwd.parent))
print(os.getcwd())

/home/montello/works/ermes/flood-delineation/src/flood-segmentation


In [3]:
from floods.datasets.flood import FloodDataset
from floods.prepare import eval_transforms, inverse_transform
from tqdm import tqdm

In [4]:
dataset_cls = FloodDataset
mean = dataset_cls.mean()[:2]
std = dataset_cls.std()[:2]

test_transform = eval_transforms(mean=mean,
                                    std=std,
                                    clip_max=30,
                                    clip_min=-30)
# create the test dataset
test_dataset = dataset_cls(path=Path('/mnt/data1/projects/shelter/flood/tiled'),
                            subset="test",
                            include_dem=False,
                            normalization=test_transform)

loader = DataLoader(test_dataset, batch_size=1, num_workers=4, pin_memory=False, shuffle=False)
invert = inverse_transform(mean=FloodDataset.mean(), std=FloodDataset.std())

In [5]:
def plot_images(output: List[np.array], titles: List[str], figsize: Tuple[int, int] = (10, 10)) -> None:
    figure(figsize=figsize, dpi=80)
    """Plot a list of images with titles"""
    for j in range(len(output)):
        plt.subplot(1, len(output), j + 1), plt.imshow(output[j], 'gray')
        plt.title(titles[j])
        plt.xticks([]), plt.yticks([])
    plt.show()


In [6]:
def minmaxscaler_image(x: np.array, min: float, max: float) -> np.array:
    return (((x - min) / (max - min))*255).astype(np.uint8)

In [10]:
def preprocess_images(images: torch.Tensor) -> np.array:
    images = images.numpy()

    vv_img = images[0, :, :]
    vh_img = images[1, :, :]

    vv_img = minmaxscaler_image(vv_img, vv_img.min(), vv_img.max())
    vh_img = minmaxscaler_image(vh_img, vh_img.min(), vh_img.max())

    vv_img = np.log10(1+ vv_img + np.finfo(np.float16).eps)
    vh_img = np.log10(1+ vh_img + np.finfo(np.float16).eps)
    
    vv_img = minmaxscaler_image(vv_img, vv_img.min(), vv_img.max())
    vh_img = minmaxscaler_image(vh_img, vh_img.min(), vh_img.max())

    return vv_img, vh_img

In [26]:
def compute_multiple_thresholds(vv: np.array, vh: np.array) -> np.array:

    # Mean thresholding
    vv_th_mean = cv.adaptiveThreshold(vv_img,255,cv.ADAPTIVE_THRESH_MEAN_C,\
                cv.THRESH_BINARY,11,2)
    vh_th_mean = cv.adaptiveThreshold(vh_img,255,cv.ADAPTIVE_THRESH_MEAN_C,\
                cv.THRESH_BINARY,11,2)

    # Gaussian thresholding
    vv_th_gauss = cv.adaptiveThreshold(vv_img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv.THRESH_BINARY,11,2)
    vh_th_gauss = cv.adaptiveThreshold(vh_img,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv.THRESH_BINARY,11,2)

    # Otsu's thresholding
    _, otsu_vv = cv.threshold(vv_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    _, otsu_vh = cv.threshold(vh_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # Otsu's thresholding after Gaussian filtering
    blur_vv = cv.GaussianBlur(vv_img, (5, 5), 0)
    blur_vh = cv.GaussianBlur(vh_img, (5, 5), 0)
    _, gauss_otsu_vv = cv.threshold(blur_vv, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    _, gauss_otsu_vh = cv.threshold(blur_vh, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)


    # Fix 1s and 0s
    pred_mean = (vv_th_mean * vh_th_mean)
    pred_mean = (1- pred_mean).astype(np.uint8)
    
    pred_gauss = (vv_th_gauss * vh_th_gauss)
    pred_gauss = (1- pred_gauss).astype(np.uint8)

    pred_otsu = ((otsu_vv + otsu_vh)/2).astype(np.uint8)
    pred_otsu = (np.invert(pred_otsu)/255).astype(np.uint8)

    pred_otsu_gauss = ((gauss_otsu_vv + gauss_otsu_vh)/2).astype(np.uint8)
    pred_otsu_gauss = (np.invert(pred_otsu_gauss)/255).astype(np.uint8)

    return pred_mean, pred_gauss, pred_otsu, pred_otsu_gauss

In [ ]:
titles = ['mask', 'vv', 'vh', 'pred_mean_vv_vh', 'pred_gaussian_vv_vh', 'pred_otsu_vv_vh', 'pred_otsu_gaussian_vv_vh']

for i, (images, label) in tqdm(enumerate(loader)):
    
    label = label.squeeze(0)
    mask = (label.flatten() != 255)
    images = images.squeeze(0)

    vv_img, vh_img = preprocess_images(images)
    pred_mean, pred_gauss, pred_otsu, pred_otsu_gauss = compute_multiple_thresholds(vv_img, vh_img)

    output = [label, vv_img, vh_img, pred_mean, pred_gauss, pred_otsu, pred_otsu_gauss]

    plot_images(output, titles, (16,20))
    
    if(i == 4):
        break


In [ ]:
titles = ['mask', 'vv', 'vh', 'pred_mean_vv_vh', 'pred_gaussian_vv_vh', 'pred_otsu_vv_vh', 'pred_otsu_gaussian_vv_vh']

prec_scores = np.zeros(test_dataset.__len__())
recall_scores = np.zeros(test_dataset.__len__())
f1_scores = np.zeros(test_dataset.__len__())
bg_f1_scores = np.zeros(test_dataset.__len__())
iou_scores = np.zeros(test_dataset.__len__())
bg_iou_scores = np.zeros(test_dataset.__len__())

for i, (images, label) in tqdm(enumerate(loader)):

    label = label.squeeze(0)
    mask = (label.flatten() != 255)
    images = images.squeeze(0)

    vv_img, vh_img = preprocess_images(images)
    pred_mean, pred_gauss, pred_otsu, pred_otsu_gauss = compute_multiple_thresholds(vv_img, vh_img)

    to_save = (pred_otsu_gauss * 255).astype(np.uint8)
    plt.imsave(f"/home/montello/works/ermes/flood-delineation/src/flood-segmentation/outputs/otsu_img_saves/{i}.png",
               to_save,
               cmap='gray')

    label = label.flatten()[mask]
    pred_otsu_gauss = pred_otsu_gauss.flatten()[mask]
    pred_otsu_gauss[pred_otsu_gauss > 0] = 1

    prec_scores[i] = precision_score(y_true=label, y_pred=pred_otsu_gauss)
    recall_scores[i] = recall_score(y_true=label, y_pred=pred_otsu_gauss)
    f1_scores[i] = f1_score(y_true=label, y_pred=pred_otsu_gauss)
    bg_f1_scores[i] = f1_score(y_true=label, y_pred=pred_otsu_gauss, pos_label=0)
    iou_scores[i] = jaccard_score(y_true=label, y_pred=pred_otsu_gauss)
    bg_iou_scores[i] = jaccard_score(y_true=label, y_pred=pred_otsu_gauss, pos_label=0)


print(f"precision: {np.mean(prec_scores)}")
print(f"recall: {np.mean(recall_scores)}")
print(f"f1: {np.mean(f1_scores)}")
print(f"bg_f1: {np.mean(bg_f1_scores)}")
print(f"iou: {np.mean(iou_scores)}")
print(f"bg_iou: {np.mean(bg_iou_scores)}")